In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import sys
import os
import xarray as xr
sys.path.append('/home/cucchi/phd/devel/phd/')

In [8]:
DATA_FORCING_PATH = '/home/cucchi/phd/data/sim/lorenz96/rk4/DF_8_0.5_0/'

In [9]:
data_noforcing = xr.open_dataarray('/home/cucchi/phd/data/sim/lorenz96/rk4/CF_8/sim_lorenz96_rk4_CF_8_one_00000.nc')

In [10]:
data_forcing = []
for i in range(0, 10):
    data_forcing.append(xr.open_dataarray(
        os.path.join(DATA_FORCING_PATH,'sim_lorenz96_rk4_DF_8_0.5_0_one_{:05}.nc'.format(i))
    ))

In [13]:
data_forcing[1]

<xarray.DataArray 'var' (time_step: 10000, node: 1)>
array([[-4.846027],
       [-4.569461],
       [-4.289452],
       ...,
       [ 1.242367],
       [ 1.358312],
       [ 1.486794]], dtype=float32)
Coordinates:
  * node       (node) int32 0
  * time_step  (time_step) int32 0 1 2 3 4 5 6 ... 9994 9995 9996 9997 9998 9999
Attributes:
    system:              Lorenz 96
    integration_method:  Runge-Kutta 4th order
    integration_step:    0.01
    forcing:             Delta Forcing (8+0.5 at t=0)
    created:             2019-05-03 15:51:52.205829
    time_step_0_real:    10000